**Installing PyTorch , IQA models**

In [ ]:
!pip install pyiqa
!git clone https://github.com/chaofengc/IQA-PyTorch.git
%cd IQA-PyTorch
!pip install -r requirements.txt
!python setup.py develop
!pwd


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.0/248.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 40.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylin

Importing **libraries** and uploading the AI Images Dataset

In [ ]:

from google.colab import files
import zipfile
import os

# Upload zip file
uploaded = files.upload()

# Assuming the zip file name is known
zip_file_name = list(uploaded.keys())[0]

# Create a directory for the input data
input_data_dir = 'input_data'
os.makedirs(input_data_dir, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(input_data_dir)

print(f"Extracted contents to {input_data_dir}")


Saving Objective_dataset.zip to Objective_dataset.zip
Extracted contents to input_data


**Importing the Neural Network Models and Saving the Prediction**

In [ ]:
import pyiqa
import torch
import json

# List all available metrics
print(pyiqa.list_models())

# Determine device (GPU if available, otherwise CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Create PI metric
pi_metric = pyiqa.create_metric('pi', device=device)

# Check if lower score is better
print(pi_metric.lower_better)

# Define output directory
output_dir = 'output_result'
os.makedirs(output_dir, exist_ok=True)

# Define metric name for the folder
metric_name = 'pi'
metric_output_dir = os.path.join(output_dir, metric_name)
os.makedirs(metric_output_dir, exist_ok=True)

# Process each subfolder in the input_data directory
for subfolder in os.listdir(input_data_dir):
    subfolder_path = os.path.join(input_data_dir, subfolder)
    if os.path.isdir(subfolder_path):
        subfolder_output_dir = os.path.join(metric_output_dir, subfolder)
        os.makedirs(subfolder_output_dir, exist_ok=True)

        # Initialize list to store results
        results = []

        # Process each image in the subfolder
        for file in os.listdir(subfolder_path):
            if file.endswith(('.jpg', '.jpeg', '.png')):  # Adjust file extensions as needed
                image_path = os.path.join(subfolder_path, file)

                try:
                    # Compute PI score for the image
                    score = pi_metric(image_path).item()  # Get the score as a Python float

                    # Extract image ID (assuming filename is in format 'pXX.png')
                    image_id = os.path.splitext(file)[0]

                    # Append result to list
                    results.append({
                        "image_id": image_id,
                        "mean_score_prediction": score
                    })

                    print(f"Processed {file} in {subfolder} and added to results.")

                except Exception as e:
                    print(f"Error processing {file} in {subfolder}: {str(e)}")

        # Save results to JSON file in the subfolder's output directory
        output_file = os.path.join(subfolder_output_dir, f'{subfolder}_results.json')
        with open(output_file, 'w') as f:
            json.dump(results, f, indent=2)

        print(f"Results for {subfolder} saved to {output_file}")

['ahiq', 'arniqa', 'arniqa-clive', 'arniqa-csiq', 'arniqa-flive', 'arniqa-kadid', 'arniqa-koniq', 'arniqa-live', 'arniqa-spaq', 'arniqa-tid', 'brisque', 'ckdn', 'clipiqa', 'clipiqa+', 'clipiqa+_rn50_512', 'clipiqa+_vitL14_512', 'clipscore', 'cnniqa', 'cw_ssim', 'dbcnn', 'dists', 'entropy', 'fid', 'fsim', 'gmsd', 'hyperiqa', 'ilniqe', 'inception_score', 'laion_aes', 'liqe', 'liqe_mix', 'lpips', 'lpips-vgg', 'mad', 'maniqa', 'maniqa-kadid', 'maniqa-koniq', 'maniqa-pipal', 'ms_ssim', 'musiq', 'musiq-ava', 'musiq-koniq', 'musiq-paq2piq', 'musiq-spaq', 'nima', 'nima-koniq', 'nima-spaq', 'nima-vgg16-ava', 'niqe', 'nlpd', 'nrqm', 'paq2piq', 'pi', 'pieapp', 'psnr', 'psnry', 'qalign', 'ssim', 'ssimc', 'stlpips', 'stlpips-vgg', 'topiq_fr', 'topiq_fr-pipal', 'topiq_iaa', 'topiq_iaa_res50', 'topiq_nr', 'topiq_nr-face', 'topiq_nr-flive', 'topiq_nr-spaq', 'tres', 'tres-flive', 'tres-koniq', 'unique', 'uranker', 'vif', 'vsi', 'wadiqam_fr', 'wadiqam_nr']
Downloading: "https://github.com/chaofengc/IQA-

100%|██████████| 66.2M/66.2M [00:00<00:00, 76.4MB/s]


Downloading: "https://github.com/chaofengc/IQA-PyTorch/releases/download/v0.1-weights/niqe_modelparameters.mat" to /root/.cache/torch/hub/checkpoints/niqe_modelparameters.mat



100%|██████████| 8.15k/8.15k [00:00<00:00, 19.7MB/s]


True
Processed p12.jpg in BingImage_Creator and added to results.
Processed p20.jpg in BingImage_Creator and added to results.
Processed p07.jpg in BingImage_Creator and added to results.
Processed p18.jpg in BingImage_Creator and added to results.
Processed p16.jpg in BingImage_Creator and added to results.
Processed p02.jpg in BingImage_Creator and added to results.
Processed p27.jpg in BingImage_Creator and added to results.
Processed p22.jpg in BingImage_Creator and added to results.
Processed p08.jpg in BingImage_Creator and added to results.
Processed p09.jpg in BingImage_Creator and added to results.
Processed p10.jpg in BingImage_Creator and added to results.
Processed p04.jpg in BingImage_Creator and added to results.
Processed p26.jpg in BingImage_Creator and added to results.
Processed p23.jpg in BingImage_Creator and added to results.
Processed p25.jpg in BingImage_Creator and added to results.
Processed p15.jpg in BingImage_Creator and added to results.
Processed p30.jpg i

In [ ]:

import pyiqa
import torch
import json

# List all available metrics
print(pyiqa.list_models())

# Determine device (GPU if available, otherwise CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Create TReS metric
tres_metric = pyiqa.create_metric('tres', device=device)

# Check if lower score is better
print(tres_metric.lower_better)



# Define output directory
output_dir = 'output_result'
os.makedirs(output_dir, exist_ok=True)

# Define metric name for the folder
metric_name = 'tres'
metric_output_dir = os.path.join(output_dir, metric_name)
os.makedirs(metric_output_dir, exist_ok=True)

# Process each subfolder in the input_data directory
for subfolder in os.listdir(input_data_dir):
    subfolder_path = os.path.join(input_data_dir, subfolder)
    if os.path.isdir(subfolder_path):
        subfolder_output_dir = os.path.join(metric_output_dir, subfolder)
        os.makedirs(subfolder_output_dir, exist_ok=True)

        # Initialize list to store results
        results = []

        # Process each image in the subfolder
        for file in os.listdir(subfolder_path):
            if file.endswith(('.jpg', '.jpeg', '.png')):  # Adjust file extensions as needed
                image_path = os.path.join(subfolder_path, file)

                try:
                    # Compute TReS score for the image
                    score = tres_metric(image_path).item()  # Get the score as a Python float

                    # Extract image ID (assuming filename is in format 'pXX.png')
                    image_id = os.path.splitext(file)[0]

                    # Append result to list
                    results.append({
                        "image_id": image_id,
                        "mean_score_prediction": score
                    })

                    print(f"Processed {file} in {subfolder} and added to results.")

                except Exception as e:
                    print(f"Error processing {file} in {subfolder}: {str(e)}")

        # Save results to JSON file in the subfolder's output directory
        output_file = os.path.join(subfolder_output_dir, f'{subfolder}_results.json')
        with open(output_file, 'w') as f:
            json.dump(results, f, indent=2)

        print(f"Results for {subfolder} saved to {output_file}")



['ahiq', 'arniqa', 'arniqa-clive', 'arniqa-csiq', 'arniqa-flive', 'arniqa-kadid', 'arniqa-koniq', 'arniqa-live', 'arniqa-spaq', 'arniqa-tid', 'brisque', 'ckdn', 'clipiqa', 'clipiqa+', 'clipiqa+_rn50_512', 'clipiqa+_vitL14_512', 'clipscore', 'cnniqa', 'cw_ssim', 'dbcnn', 'dists', 'entropy', 'fid', 'fsim', 'gmsd', 'hyperiqa', 'ilniqe', 'inception_score', 'laion_aes', 'liqe', 'liqe_mix', 'lpips', 'lpips-vgg', 'mad', 'maniqa', 'maniqa-kadid', 'maniqa-koniq', 'maniqa-pipal', 'ms_ssim', 'musiq', 'musiq-ava', 'musiq-koniq', 'musiq-paq2piq', 'musiq-spaq', 'nima', 'nima-koniq', 'nima-spaq', 'nima-vgg16-ava', 'niqe', 'nlpd', 'nrqm', 'paq2piq', 'pi', 'pieapp', 'psnr', 'psnry', 'qalign', 'ssim', 'ssimc', 'stlpips', 'stlpips-vgg', 'topiq_fr', 'topiq_fr-pipal', 'topiq_iaa', 'topiq_iaa_res50', 'topiq_nr', 'topiq_nr-face', 'topiq_nr-flive', 'topiq_nr-spaq', 'tres', 'tres-flive', 'tres-koniq', 'unique', 'uranker', 'vif', 'vsi', 'wadiqam_fr', 'wadiqam_nr']


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 135MB/s]


Downloading: "https://github.com/chaofengc/IQA-PyTorch/releases/download/v0.1-weights/tres_koniq-f0502926.pth" to /root/.cache/torch/hub/checkpoints/tres_koniq-f0502926.pth



100%|██████████| 582M/582M [00:09<00:00, 67.2MB/s]


Loading pretrained model TReS from /root/.cache/torch/hub/checkpoints/tres_koniq-f0502926.pth
False
Processed p12.jpg in BingImage_Creator and added to results.
Processed p20.jpg in BingImage_Creator and added to results.
Processed p07.jpg in BingImage_Creator and added to results.
Processed p18.jpg in BingImage_Creator and added to results.
Processed p16.jpg in BingImage_Creator and added to results.
Processed p02.jpg in BingImage_Creator and added to results.
Processed p27.jpg in BingImage_Creator and added to results.
Processed p22.jpg in BingImage_Creator and added to results.
Processed p08.jpg in BingImage_Creator and added to results.
Processed p09.jpg in BingImage_Creator and added to results.
Processed p10.jpg in BingImage_Creator and added to results.
Processed p04.jpg in BingImage_Creator and added to results.
Processed p26.jpg in BingImage_Creator and added to results.
Processed p23.jpg in BingImage_Creator and added to results.
Processed p25.jpg in BingImage_Creator and add